# Home Credit Default Risk- Encyclopedic Data Analysis and Exploration using Python ✍

### This note book is work in progress. Stay tuned for more indepth analysis and insights

## Give me six hours to chop down a tree and i will spend the first four sharpening the axe - Abraham Lincon.
#### This makes perfect sense to me in the area of machine learning and statistical modeling and this is exactly what i would do when i build a model. In fact, i would spend five and half in shapening the axe (data prepartion) .

#### In the advent of various machine learning algorithms like Random Forest, XG Boost etc.. you don't have to spend much time in variable selection and treatment, but it's always fun to know what and how you are going to process the features and for the fact that, your results are directly proportional to quality of  your input variables. 


#### I'm going to drive this notebook in terms of various steps involved in the process of building a good model and i see it more useful for beginner to intermediate data scientists

#### As always, any comments / suggestions are much appreciated :)

## Index:
- <a href='#pb'>1. Problem Statement</a>
- <a href='#lb'>2. Libraries</a>
- <a href='#dl'>3. Get the Data</a>
    - <a href='#lk'>3.1 Quick look at the data</a>
- <a href='#dp'>4. Data Exploration</a>
     - <a href='#vi'>4.1 Variable Identification</a>
     - <a href='#uv'>4.2 Univariate Analysis</a>
- <a href='#mv'>5. Missing Value Treatment</a>
- <a href='#ot'>6. Outlier Treatment</a>


## <a id='pb'>1. Problem Statement</a>

  #### As per the description mentioned [here](https://www.kaggle.com/c/home-credit-default-risk),  we are going to build a model for the credit invisibles (whose credit history was very limited / unavailable with the bureau) to estimater whether the given customer will be able to pay the loan installements on time or not. For this, we are going to depend mostly on the interal data that the bank collected at the time of application / alternate data sources than the treditional bureau data. 
  #### Target variable (1 - client with payment difficulties: he/she had late payment more than X days on at least one of the first Y installments of the loan in our sample, 0 - all other cases)
 

## <a id='lb'>2. Libraries</a>

In [190]:
import numpy as np
import pandas as pd   
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
pd.set_option('display.max_row',10000)
pd.set_option('display.max_columns',150)

## <a id='dl'>3. Get the Data</a>

In [191]:
app_train = pd.read_csv('../input/application_train.csv')
app_test=pd.read_csv('../input/application_test.csv')
bureau = pd.read_csv('../input/bureau.csv')
bureau_bal = pd.read_csv('../input/bureau_balance.csv')
pos_cash = pd.read_csv('../input/POS_CASH_balance.csv')
cc_bal = pd.read_csv('../input/credit_card_balance.csv')
app_prev = pd.read_csv('../input/previous_application.csv')
instal_pay = pd.read_csv('../input/installments_payments.csv')

## <a id='lk'>3.1 Quick look at the data</a>

In [192]:
app_train.shape

In [193]:
app_train.head()

In [40]:
len(app_train['SK_ID_CURR'].unique().tolist())

In [194]:
app_test.shape

In [195]:
len(app_test['SK_ID_CURR'].unique().tolist())

In [196]:
bureau.shape

In [197]:
bureau.head()

In [198]:
len(bureau['SK_ID_CURR'].unique().tolist())

In [199]:
len(bureau['SK_ID_BUREAU'].unique().tolist())

In [200]:
pos_cash.shape

In [201]:
len(pos_cash['SK_ID_CURR'].unique().tolist())

In [202]:
cc_bal.shape

In [203]:
len(cc_bal['SK_ID_CURR'].unique().tolist())

As mentioend in the data description,  excpet for the train and test datasets, all other dataset have duplicates/repeated entries by the ID variable. So need to be carefull when we join the datasets

### Lets have a peek at the distribution of the target variable in the train dataset, 

In [204]:
bad_dist = app_train['TARGET'].value_counts().reset_index()
bad_dist.columns = ['val','count']
bad_dist['percent'] = (bad_dist['count']/app_train.shape[0])*100
bad_dist

In [205]:
import matplotlib.pyplot as plt
plt.figure()
plt.bar(app_train['TARGET'].value_counts().index,app_train['TARGET'].value_counts());


#### It's an example of imbalanced data with ~92% good and 8% bad observations. Make sense to look at ROC as the evaluation metric than Accuracy

#### Lets combine the train and test data set for preprocessing

In [206]:
train_id = app_train['SK_ID_CURR']
test_id = app_test['SK_ID_CURR']
train_y = app_train['TARGET']
app_train['key'] = 'train'
app_train.drop(['SK_ID_CURR','TARGET'],axis=1,inplace=True)
app_test.drop(['SK_ID_CURR'],axis=1,inplace=True)
app_test['key']='test'
data=pd.concat((app_train,app_test)).reset_index(drop=True)


In [207]:
data.shape

## <a id='vi'>4. Variable Identification</a>

Lets look at the variable type, data type and also the variable category (categorical/contineous). We will convert the existing variables type to other types if it is necessary

In [208]:
bureau.info()

In [209]:
#Randomly looking at the distribution of the variables that i'm curious about
bureau.groupby(['CREDIT_CURRENCY'])['CREDIT_CURRENCY'].count()

In [210]:
bureau_bal.info()

In [211]:
bureau_bal.groupby(['STATUS'])['STATUS'].count()

In [212]:
instal_pay.info()

In [213]:
app_prev.info()

In [214]:
cc_bal.info()

In [215]:
pos_cash.info()

Looks like the variables are assigned correctly and we don't have to change anything

## <a id='uv'>4.2 Univariate Analysis </a>

#### Lets look at the central tendency for continuous variale and the frequncy tables for categorical variables. Before diving into it, let's seggreate the continuous and categorical variables into separate datasets for convenience

In [216]:
cat_data=data.select_dtypes(include=["O"])

In [217]:
num_data = data.select_dtypes(include=["number"])

In [218]:
num_data.shape

In [219]:
cat_data.shape

####  So we have 104 numerical columns and 17 categorical columns, which matches to the total 121 columns in the train data. 

In [220]:
cat_data.describe()

#### The above output gives you an indepth information about all categorical variables. It gives you the total count (we can get the missing count from this, if any), unique values in each column, the value repeated the most number of times and the number of time the top value was repeated. It's a very good insight to start with. From this, you can also see if the frequncy value in the about table is more than 90% of the count, then the variable is very unlikely to give us any information about the target variable. Hence we can delete the variable. Anyhow, let's do it in in a while after making sure of that once again. As of now, i see the variables 'NAME_CONTRACT_TYPE' and 'NAME_HOUSING_TYPE' are the one's that have the same value for 90% of the observations

In [221]:
pd.set_option('display.float_format',lambda x: '%.1f' % x)
num_data.describe()

#### This is way too much info and should spend significant amount of time to understand the variables and their distirbution. All the days fields here have -ve values as they are calculated based on the appliation date (howmany days ago from the application date). if you see the same values for 25th, 50th and 75th percentile, then ideally that variable wouldn't give much information about the target variable. 

#### From the above distribution, you can see there are outliers for quite a few variables and will treat them in a while. Let's look at the missing values

## <a id='mv'>5. Missing Value Treatment </a>

In [222]:
missing = num_data.isnull().sum(axis=0).reset_index()
missing.columns=['column_name','missing_count']
percent = ((num_data.isnull().sum(axis=0)/num_data.shape[0])*100).reset_index()
percent.columns=['variable','percent']
missing_data = pd.concat([missing,percent['percent']],axis=1)
missing_data=missing_data.ix[missing_data['missing_count']>0].sort_values(by='missing_count')

In [223]:
missing_data

#### We have handful number of variables with missing values. I'm going to delete all the variables with more than 45% of the values are missing

In [224]:
missing_col=missing_data['column_name'][missing_data['percent']> 45].tolist()
for i in missing_col:
    num_data.drop([i],axis=1,inplace=True)

In [84]:
num_data.shape

By looking at the data dictionary, i'm going to impute the missing values using the most appropirate method that i think would make sense

In [105]:
num_data['DAYS_LAST_PHONE_CHANGE']= num_data['DAYS_LAST_PHONE_CHANGE'].fillna(num_data['DAYS_LAST_PHONE_CHANGE'].mode()[0])
num_data['CNT_FAM_MEMBERS']= num_data['CNT_FAM_MEMBERS'].fillna(num_data['CNT_FAM_MEMBERS'].mode()[0])
num_data['AMT_ANNUITY']= num_data['AMT_ANNUITY'].fillna(num_data['AMT_ANNUITY'].mean())
num_data['AMT_GOODS_PRICE']= num_data['AMT_GOODS_PRICE'].fillna(0)
num_data['EXT_SOURCE_2']= num_data['EXT_SOURCE_2'].fillna(num_data['EXT_SOURCE_2'].mean())
num_data['DEF_60_CNT_SOCIAL_CIRCLE']= num_data['DEF_60_CNT_SOCIAL_CIRCLE'].fillna(num_data['DEF_60_CNT_SOCIAL_CIRCLE'].mode()[0])
num_data['OBS_60_CNT_SOCIAL_CIRCLE']= num_data['OBS_60_CNT_SOCIAL_CIRCLE'].fillna(num_data['OBS_60_CNT_SOCIAL_CIRCLE'].mode()[0])
num_data['DEF_30_CNT_SOCIAL_CIRCLE']= num_data['DEF_30_CNT_SOCIAL_CIRCLE'].fillna(num_data['DEF_30_CNT_SOCIAL_CIRCLE'].mode()[0])
num_data['OBS_30_CNT_SOCIAL_CIRCLE']= num_data['OBS_30_CNT_SOCIAL_CIRCLE'].fillna(num_data['OBS_30_CNT_SOCIAL_CIRCLE'].mode()[0])
num_data['AMT_REQ_CREDIT_BUREAU_MON']= num_data['AMT_REQ_CREDIT_BUREAU_MON'].fillna(num_data['AMT_REQ_CREDIT_BUREAU_MON'].mode()[0])
num_data['AMT_REQ_CREDIT_BUREAU_WEEK']= num_data['AMT_REQ_CREDIT_BUREAU_WEEK'].fillna(num_data['AMT_REQ_CREDIT_BUREAU_WEEK'].mode()[0])
num_data['AMT_REQ_CREDIT_BUREAU_DAY']= num_data['AMT_REQ_CREDIT_BUREAU_DAY'].fillna(num_data['AMT_REQ_CREDIT_BUREAU_DAY'].mode()[0])
num_data['AMT_REQ_CREDIT_BUREAU_HOUR']= num_data['AMT_REQ_CREDIT_BUREAU_HOUR'].fillna(num_data['AMT_REQ_CREDIT_BUREAU_HOUR'].mode()[0])
num_data['AMT_REQ_CREDIT_BUREAU_QRT']= num_data['AMT_REQ_CREDIT_BUREAU_QRT'].fillna(num_data['AMT_REQ_CREDIT_BUREAU_QRT'].mode()[0])
num_data['AMT_REQ_CREDIT_BUREAU_YEAR']= num_data['AMT_REQ_CREDIT_BUREAU_YEAR'].fillna(num_data['AMT_REQ_CREDIT_BUREAU_YEAR'].mode()[0])
num_data['EXT_SOURCE_3']= num_data['EXT_SOURCE_3'].fillna(num_data['EXT_SOURCE_3'].mean())

In [108]:
missing_cat = cat_data.isnull().sum(axis=0).reset_index()
missing_cat.columns=['column_name','missing_count']
percent_cat = ((cat_data.isnull().sum(axis=0)/cat_data.shape[0])*100).reset_index()
percent_cat.columns=['variable','percent']
missing_data_cat = pd.concat([missing_cat,percent_cat['percent']],axis=1)
missing_data_cat=missing_data_cat.ix[missing_data_cat['missing_count']>0].sort_values(by='missing_count')

In [109]:
missing_data_cat

In [157]:
missing_col_cat=missing_data_cat['column_name'][missing_data_cat['percent']> 45].tolist()
for i in missing_col_cat:
    cat_data.drop([i],axis=1,inplace=True)

In [158]:
cat_data.shape

In [159]:
cat_data['NAME_TYPE_SUITE']= cat_data['NAME_TYPE_SUITE'].fillna(cat_data['NAME_TYPE_SUITE'].mode()[0])

In case of missing values for occupation, we can kind of predict it from their level of eduction. Let's see the distribution of the education level where the occupation was null. 

In [125]:
cat_data['NAME_EDUCATION_TYPE'][cat_data.OCCUPATION_TYPE.isnull()].value_counts()

so most of them were with the 'Seconday / secondary special' education level. Lets look at the distribution of the occupation in this education level. From the below distribution, so most of them are Laboreres. I will replace the missing value with corresponding the secondary education level with 'Laborers' 

In [132]:
cat_data['OCCUPATION_TYPE'][cat_data['NAME_EDUCATION_TYPE']=='Secondary / secondary special'].value_counts()

In [160]:
cat_data['OCCUPATION_TYPE'][cat_data['NAME_EDUCATION_TYPE']=='Secondary / secondary special'] =  cat_data['OCCUPATION_TYPE'][cat_data['NAME_EDUCATION_TYPE']=='Secondary / secondary special'].fillna(cat_data['OCCUPATION_TYPE'][cat_data['NAME_EDUCATION_TYPE']=='Secondary / secondary special'].mode()[0])
cat_data['OCCUPATION_TYPE'][cat_data['NAME_EDUCATION_TYPE']=='Higher education'] =  cat_data['OCCUPATION_TYPE'][cat_data['NAME_EDUCATION_TYPE']=='Higher education'].fillna(cat_data['OCCUPATION_TYPE'][cat_data['NAME_EDUCATION_TYPE']=='Higher education'].mode()[0])
cat_data['OCCUPATION_TYPE'][cat_data['NAME_EDUCATION_TYPE']=='Incomplete higher'] = cat_data['OCCUPATION_TYPE'][cat_data['NAME_EDUCATION_TYPE']=='Incomplete higher'].fillna(cat_data['OCCUPATION_TYPE'][cat_data['NAME_EDUCATION_TYPE']=='Incomplete higher'].mode()[0])
cat_data['OCCUPATION_TYPE'][cat_data['NAME_EDUCATION_TYPE']=='Lower secondary'] = cat_data['OCCUPATION_TYPE'][cat_data['NAME_EDUCATION_TYPE']=='Lower secondary'].fillna(cat_data['OCCUPATION_TYPE'][cat_data['NAME_EDUCATION_TYPE']=='Lower secondary'].mode()[0])
cat_data['OCCUPATION_TYPE'][cat_data['NAME_EDUCATION_TYPE']=='Academic degree'] = cat_data['OCCUPATION_TYPE'][cat_data['NAME_EDUCATION_TYPE']=='Academic degree'].fillna(cat_data['OCCUPATION_TYPE'][cat_data['NAME_EDUCATION_TYPE']=='Academic degree'].mode()[0])

## <a id='ot'>6. Outlier Treatment </a>

So we're done with the missing value imputation. Let's clip the outliers.

In [162]:
for i in num_data.columns:
    ll,ul = np.percentile(num_data[i],[1,99])
    num_data[i].ix[num_data[i]>ul] = ul
    num_data[i].ix[num_data[i] < ll]=ll
    

In [163]:
num_data.describe()

We've to repeat the same process of missing and outlier treatment for other datasets aswell.'

## Visualisation and Feature engineering in the next update

# Stay tuned for more and don't forget to upvote if you like it.
